**Dataset Information**: Dataset is extracted from the electric current drive signals. The drive contains intact as well as defective components. Therefore, dataset has 11 classes based on the condition of the components. Aim is, to predict the correct component condition based on the input variables using **Deep Learning** technique. Tools used: **Keras TensorFlow** 

**Dataset Rights**: This dataset has been taken from "University of California Irvine Machine Learning Repository" for the knowledge purpose and all the rights for this dataset are reserved by them. For more details like content of the dataset, owner of the dataset and reference research paper, please refer the following link: https://archive.ics.uci.edu/ml/datasets/Dataset+for+Sensorless+Drive+Diagnosis

In [89]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, Conv2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
#from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder

In [77]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

Step 2. Import and print data

In [4]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df = pd.read_csv(r'N:\\connectxactions\\full_actions.csv',index_col=0)
# Print first 10 samples
print(df.head(10))

   1  2  3  4  5  6  7  8  9  10  ...  36  37  38  39  40  41  42  target  \
0  0  0  0  0  0  0  0  0  0   0  ...   0   0   0   1   0   0   0       1   
1  0  0  0  0  0  0  0  0  0   0  ...   0   2   0   1   0   0   0       3   
2  0  0  0  0  0  0  0  0  0   0  ...   0   2   0   1   1   0   0       2   
3  0  0  0  0  0  0  0  0  0   0  ...   0   2   2   1   1   0   0       4   
4  0  0  0  0  0  0  0  0  0   0  ...   0   2   2   1   1   0   0       2   
5  0  0  0  0  0  0  0  0  0   0  ...   0   2   2   1   1   1   0       6   
6  0  0  0  0  0  0  0  0  0   0  ...   0   2   2   1   1   1   2       3   
7  0  0  0  0  0  0  0  0  0   0  ...   0   2   2   1   1   1   2       3   
8  0  0  0  1  0  0  0  0  0   0  ...   0   2   2   1   1   1   2       4   
9  0  0  0  1  0  0  0  0  0   0  ...   0   2   2   1   1   1   2       6   

   winner  tick  
0     2.0     1  
1     2.0     1  
2     2.0     1  
3     2.0     1  
4     2.0     1  
5     2.0     1  
6     2.0     1  
7     2.

In [5]:
df

,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,target,winner,tick
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,2.0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,1,0,0,0,3,2.0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,2,0,1,1,0,0,2,2.0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,2,2,1,1,0,0,4,2.0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,2,2,1,1,0,0,2,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067337,2,0,0,0,0,0,0,2,0,0,...,2,0,1,1,0,1,1,1,1.0,1
1067338,2,0,0,0,0,0,0,2,0,0,...,2,1,1,1,0,1,1,1,1.0,1
1067339,2,0,0,0,0,0,0,2,0,0,...,2,1,1,1,0,1,1,3,1.0,1
1067340,2,0,0,0,0,0,0,2,2,0,...,2,1,1,1,0,1,1,3,1.0,1


In [120]:
df.loc[df['target'] == 3, 'target_three'] = 1
df.loc[df['target'] != 3, 'target_three'] = 0

In [121]:
df

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,target,winner,tick,target_three
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,2.0,1,0.0
1,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,0,3,2.0,1,1.0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,1,1,0,0,2,2.0,1,0.0
3,0,0,0,0,0,0,0,0,0,0,...,2,2,1,1,0,0,4,2.0,1,0.0
4,0,0,0,0,0,0,0,0,0,0,...,2,2,1,1,0,0,2,2.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067337,2,0,0,0,0,0,0,2,0,0,...,0,1,1,0,1,1,1,1.0,1,0.0
1067338,2,0,0,0,0,0,0,2,0,0,...,1,1,1,0,1,1,1,1.0,1,0.0
1067339,2,0,0,0,0,0,0,2,0,0,...,1,1,1,0,1,1,3,1.0,1,1.0
1067340,2,0,0,0,0,0,0,2,2,0,...,1,1,1,0,1,1,3,1.0,1,1.0


Column indices 0 to 47 are input variables (total 48 columns). Column index 48 is target column that contains 11 different classes (1 column). 

Step 3. Data pre-processing

In [6]:
# Check missing values
print(df.isna().sum())

1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
31        0
32        0
33        0
34        0
35        0
36        0
37        0
38        0
39        0
40        0
41        0
42        0
target    0
winner    0
tick      0
dtype: int64


No missing values. 

In [7]:
# Remove missing values IF AVAILABLE and print first 10 samples
# df = df.dropna()
# print(df.head(10))
# print(df.shape)


In [127]:
# Divide data into features X and target (Classes) Y
columns = [str(item) for item in range(1, 43)]
columns.append('winner')
X = df.loc[:,columns]
Y = df.loc[:,'target_three']
print(X.shape)
print(Y.shape)

(1067342, 43)
(1067342,)


In [128]:
# Statistical summary of the variables
print(X.describe())

                  1             2             3             4             5  \
count  1.067342e+06  1.067342e+06  1.067342e+06  1.067342e+06  1.067342e+06   
mean   1.332113e-01  1.483845e-01  1.722859e-01  3.719604e-01  1.704449e-01   
std    4.500658e-01  4.668083e-01  5.074080e-01  7.179318e-01  5.069355e-01   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
50%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
75%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
max    2.000000e+00  2.000000e+00  2.000000e+00  2.000000e+00  2.000000e+00   

                  6             7             8             9            10  \
count  1.067342e+06  1.067342e+06  1.067342e+06  1.067342e+06  1.067342e+06   
mean   1.330548e-01  9.542677e-02  1.975496e-01  2.402613e-01  2.591194e-01   
std    4.424060e-01  3.896117e-01  5.342196e-01  5.

Scale of all the variables is different. Therefore, feature scaling is important.    

In [129]:
# Check for class imbalance
print(df.groupby(Y).size())

target_three
0.0    879227
1.0    188115
dtype: int64


Since all the classes have same sample size, there is no class imbalance. 

In [130]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
X = pd.DataFrame(X)

To train the Neural Network, single target column must be converted into one hot encoded fomat. For more details, visit this link: https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [131]:
# Convert target Y to one hot encoded Y for Neural Network
Y = pd.get_dummies(Y)
# If target is in string form, use following code:
# First encode target values as integers from string
# Then perform one hot encoding
# encoder = LabelEncoder()
# encoder.fit(Y)
# Y = encoder.transform(Y)
# Y = np_utils.to_categorical(Y)

In [132]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X = X.values
Y = Y.values

Step 4. Define Neural Network Model 

Two hidden layers are defined with "Rectified Linear Unit" (relu) and 15 neurons each. Furthermore, this is a multi-class classification problem and there are total 11 target clsses, therefore "softmax" activation function and 11 neurons are used in the output layer. For hidden layers, the number of neurons should be in between the input data dimension and the output data dimension. In this case, the input data has 48 variable columns and output classes are 11. Therefore, the number of neurons for the hidden layer should be in between 11 and 48. You can try different values for the number of neurons as well as different number of hidden layers.  

In [138]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = Sequential()
    #model.add(Dense(150, input_dim = 43, activation = 'relu')) # Rectified Linear Unit Activation Function
    #model.add(Dense(7, activation = 'relu'))
    #model.add(Dropout(.2))
    #model.add(Dense(150, activation = 'relu'))
    #model.add(Dense(7, activation = 'softmax')) # Softmax for multi-class classification
    model.add(Dense(43, activation= 'tanh'))
    model.add(Dense(7, activation='softmax'))
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model_binary():
    # Create model here
    model = Sequential()
    #model.add(Dense(150, input_dim = 43, activation = 'relu')) # Rectified Linear Unit Activation Function
    #model.add(Dense(7, activation = 'relu'))
    #model.add(Dropout(.2))
    #model.add(Dense(150, activation = 'relu'))
    #model.add(Dense(7, activation = 'softmax')) # Softmax for multi-class classification
    model.add(Dense(43, activation= 'relu'))
    model.add(Dense(2, activation='sigmoid'))
    # Compile model here
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

Input dimension (input_dim) is 48, because the input variable columns are 48. It will change as per the dimension of the input variables.

**Note**: If you use only one hidden layer, then it will be the case of simple Neural Network problem. But, if you use more than one hidden layers for example 3, it will be considered as the deep learning problem.  

In [139]:
# Create Keras Classifier and use predefined baseline model
estimator = KerasClassifier(build_fn = baseline_model_binary, epochs = 10, batch_size = 100, verbose = 1)
# Try different values for epoch and batch size

C:\Users\ryans\AppData\Local\Temp\ipykernel_33056\4241535899.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn = baseline_model_binary, epochs = 10, batch_size = 100, verbose = 1)


Step 5. Define cross-validation and train pre-defined model 

In [140]:
# KFold Cross Validation
kfold = KFold(n_splits = 5, shuffle = True, random_state = seed)
# Try different values of splits e.g., 10

In [141]:
# Object to describe the result
#results = cross_val_score(estimator, X, Y, cv = kfold)
# Result
#print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

From the result above, the accuracy is 94% and it can be improved by techniques like feature extraction, selection and feature engineering. 

In [142]:
estimator.fit(X,Y)

Epoch 1/10
10674/10674 [==============================] - 23s 2ms/step - loss: 0.3859 - accuracy: 0.8343
Epoch 2/10
10674/10674 [==============================] - 23s 2ms/step - loss: 0.3752 - accuracy: 0.8378
Epoch 3/10
10674/10674 [==============================] - 24s 2ms/step - loss: 0.3716 - accuracy: 0.8389
Epoch 4/10
10674/10674 [==============================] - 24s 2ms/step - loss: 0.3693 - accuracy: 0.8405
Epoch 5/10
10674/10674 [==============================] - 25s 2ms/step - loss: 0.3678 - accuracy: 0.8414
Epoch 6/10
10674/10674 [==============================] - 25s 2ms/step - loss: 0.3667 - accuracy: 0.8423
Epoch 7/10
10674/10674 [==============================] - 24s 2ms/step - loss: 0.3659 - accuracy: 0.8427
Epoch 8/10
10674/10674 [==============================] - 24s 2ms/step - loss: 0.3651 - accuracy: 0.8430
Epoch 9/10
10674/10674 [==============================] - 23s 2ms/step - loss: 0.3643 - accuracy: 0.8433
Epoch 10/10
10674/10674 [==============================

In [148]:
estimator.predict([X[20].tolist()])

1/1 [==============================] - 0s 31ms/step


array([0])

In [144]:
model_name = 'test2'
estimator.model.save(f'{model_name}')
estimator.model.save(f'{model_name}.h5', save_format='h5')

INFO:tensorflow:Assets written to: test2\assets


In [145]:
import keras
model2 = keras.models.load_model(f'{model_name}.h5')

In [151]:
np.argmax(model2.predict([X[0].tolist()]))

1/1 [==============================] - 0s 32ms/step


0

In [156]:
model2.predict([X[0:40]])

2/2 [==============================] - 0s 2ms/step


array([[5.2892160e-01, 4.7118160e-01],
       [4.0977478e-01, 5.9032077e-01],
       [7.5004727e-01, 2.5013372e-01],
       [8.5294670e-01, 1.4714265e-01],
       [8.8704461e-01, 1.1299854e-01],
       [7.1436960e-01, 2.8571245e-01],
       [7.4085510e-01, 2.5918213e-01],
       [8.4983516e-01, 1.5012769e-01],
       [1.0000000e+00, 1.1300502e-13],
       [1.0000000e+00, 2.4020978e-12],
       [1.0000000e+00, 2.5670535e-11],
       [1.0000000e+00, 1.3910804e-10],
       [1.0000000e+00, 5.1826760e-10],
       [1.0000000e+00, 7.3966642e-09],
       [1.0000000e+00, 7.8278433e-09],
       [5.2054358e-01, 4.7957474e-01],
       [8.4177047e-01, 1.5828006e-01],
       [8.5836208e-01, 1.4166206e-01],
       [8.2997131e-01, 1.7005564e-01],
       [8.3171511e-01, 1.6830209e-01],
       [8.7024081e-01, 1.2976047e-01],
       [8.1399059e-01, 1.8600841e-01],
       [8.6758858e-01, 1.3240838e-01],
       [8.6227769e-01, 1.3772045e-01],
       [8.4537029e-01, 1.5466575e-01],
       [8.7407148e-01, 1.